In [1]:
import numpy as np
import pandas as pd 
import csv
import matplotlib.pyplot as plt
import seaborn as sns 

ModuleNotFoundError: No module named 'seaborn'

In [4]:
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
# load dataset
pima = pd.read_csv("pima-indians-diabetes.csv", header = None, names=col_names)
pima.head()
pima.shape
pima.isna().sum() #check for missing values and also the data types 


pregnant    0
glucose     0
bp          0
skin        0
insulin     0
bmi         0
pedigree    0
age         0
label       0
dtype: int64

In [5]:
#split dataset in features and target variable
X = pima.drop(columns = ['label']) # Features
y_label = pima['label'].values.reshape(X.shape[0],1) # Target variable

In [6]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest= train_test_split(X, y_label, test_size= 0.3)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(Xtrain)
Xtrain = sc.transform(Xtrain)
Xtest = sc.transform(Xtest)

In [ ]:
class NeuralNetwork():
    def __init__(self, layers = [8,5,1], learning_rate = 0.001, iterations = 1000):
        self.params = {}
        self.learning_rate = learning_rate
        self.iterations = iterations 
        self.loss = []
        self.sample_size = None
        self.layers = layers
        self.X = None
        self.y = None
    def init_weights(self):
        np.random.seed(1)
        self.params["W1"] = np.random.randn(self.layers[0], self.layers[1])
        self.params["b1"] = np.random.randn(self.layers[1],)
        self.params["W2"] = np.random.randn(self.layers[1], self.layers[2])
        self.params["b2"] = np.random.randn(self.layers[2],)
    def entropy_loss(self, y, yhat):
        nsample = len(y)
        loss = -1/nsample * (np.sum(np.multiply(np.log(yhat),y) + np.multiply((1-y), np.log(1-yhat))))
        return loss
    def relu(self,Z):
        return np.maximum(0,Z)
    def sigmoid(self, x):
        return 1/ (1 + np.exp(-x))
    def forward_propagation(self):
        Z1 = self.X.dot(self.params['W1']) + self.params['b1']
        A1 = self.sigmoid(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        yhat = self.sigmoid(Z2)
        loss = self.entropy_loss(self.y, yhat)
        
        self.params['Z1'] = Z1
        self.params['Z2'] = Z2
        self.params['A1'] = A1
        
        return yhat,loss
    def back_propagation(self, yhat):
        
        def dRelu(x):
            x[x<=0] = 0
            x[x>0] = 1
            return x
        def dSigmoid(x):
            return x*(1-x)
        
        dL_yhat = -(np.divide(self.y,yhat)) - np.divide((1-self.y),(1-yhat))
        dL_sig = yhat * (1-yhat)
        dL_z2 = dL_yhat * dL_sig
        
        dL_A1 = dL_z2.dot(self.params['W2'].T)
        dL_w2 = self.params['A1'].T.dot(dL_z2)
        dL_b2 = np.sum(dL_z2, axis = 0) #
        
        dL_z1 = dL_A1 * dSigmoid(self.params['Z1'])
        dL_w1 = self.X.T.dot(dL_z1)
        dL_b1 = np.sum(dL_z1, axis = 0)
        
        #update the weights and bias 
        self.params['W1'] -= self.learning_rate * dL_w1
        self.params['W2'] -= self.learning_rate * dL_w2
        self.params['b1'] -= self.learning_rate * dL_b1
        self.params['b2'] -= self.learning_rate * dL_b2
    
    def fit(self, X, y):
        # Trains the neural network using the specified data and labels
    
        self.X = X 
        self.y = y
        self.init_weights()
    
        for i in range(self.iterations):
            yhat, loss = self.forward_propagation()
            self.back_propagation(yhat)
            self.loss.append(loss)
    
    def predict(self,X):
        #Predict on a test data
        Z1 = X.dot(self.params['W1']) + self.params['b1']
        A1 = self.sigmoid(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        pred = self.sigmoid(Z2)
        return np.round(pred)
    
    def acc(self, y, yhat):
        #Calculates the accuracy between the predicted value and the truth labels
        
        acc = int(sum(y==yhat) / len(y) * 100)
        return acc
    
    def plot_loss(self):
        #Plot the loss curve 
        
        plt.plot(self.loss)
        plt.xlabel("Iterations")
        plt.ylabel("logloss")
        plt.title("Loss curve for training")
        plt.show()

In [8]:
nn = NeuralNetwork()
nn.fit(Xtrain, ytrain)
nn.plot_loss()

NameError: name 'NeuralNetwork' is not defined

In [ ]:
train_pred = nn.predict(Xtrain)
test_pred = nn.predict(Xtest)

print("Train accuracy is {}".format(nn.acc(ytrain, train_pred)))
print("Test accuracy is {}".format(nn.acc(ytest, test_pred)))